In [1]:
%load_ext autoreload
%autoreload 1

In [3]:
import pandas as pd
import numpy as np
import pickle
import sys
sys.path.append('diachronic-p2v')
%aimport preprocessing
%aimport utils
%aimport p2v

from utils import prinT

pd.options.display.max_columns = None
pd.set_option('max_colwidth', None)

In [4]:
model = p2v.P2V()

2024-11-11 23:04:09 -- start loading Mag_venue_info_df
2024-11-11 23:04:10 -- finish.
2024-11-11 23:04:10 -- start loading labeled_journal_info_df
2024-11-11 23:04:10 -- finish.


# Periodicals' VID and their names in MAG

In [15]:
model.MAG_venue_info_df[model.MAG_venue_info_df['OriginalVenue'].str.contains('Proceedings of The Royal Society B: Biological Sciences', na=False, case=False)]

,VenueID,VenueType,OriginalVenue,PublicationVolume,PublicationVolume_1950s,PublicationVolume_1960s,PublicationVolume_1970s,PublicationVolume_1980s,PublicationVolume_1990s,PublicationVolume_2000s,PublicationVolume_2010s
864,3006207977,Journal,Proceedings of The Royal Society B: Biological Sciences,20208,753,874,1201,1486,2407,4120,7045


# MAG periodicals with Scopus labels

In [248]:
venue_name = 'Web and Social Media'
model.labeled_journal_info_df[(model.labeled_journal_info_df['OriginalVenue'].str.contains(venue_name, case=False))|
(model.labeled_journal_info_df['ScopusName'].str.contains(venue_name, case=False))]

,VenueType,OriginalVenue,ScopusName,ScopusCategory,InceptionYear
VenueID,,,,,


# look up periodicals' neighbors in different decades

In [4]:
start_year_list = ['1950', '1960', '1970', '1980', '1990', '2000', '2010']
end_year_list = ['1959', '1969', '1979', '1989', '1999', '2009', '2021']

In [23]:
def display_neighbors(venue_name, start_year, end_year, k):
    tar_wv = model.load_wv(start_year, end_year, 100, 10)
    VID = model.MAG_venue_info_df[model.MAG_venue_info_df.OriginalVenue==venue_name].VenueID.values[0]
    tup_list = tar_wv.most_similar(positive=VID, topn=k)
    for tup in tup_list: 
        print (model.MAG_venue_info_df[model.MAG_venue_info_df['VenueID']==tup[0]].OriginalVenue.values[0], tup[1])

In [24]:
venue_name = 'Nature'
for i in range(0, len(start_year_list)):
    try: 
        print('------------------')
        print('find neighbors for '+venue_name+' in '+start_year_list[i]+'\'s:')
        display_neighbors(venue_name, start_year_list[i], end_year_list[i], 11)
    except KeyError:
        print("This periodical was not included in the embedding in that decade.")
        pass

------------------
find neighbors for Nature in 1950's:
2024-11-06 16:51:11 -- start loading word vectors...
2024-11-06 16:51:11 -- word vectors loaded, and its shape is: (1645, 100)
Journal of Molecular Biology 0.5745090246200562
Biochimica et Biophysica Acta 0.5562065243721008
Naturwissenschaften 0.5530214309692383
Current Science 0.5423795580863953
Research; a journal of science and its applications 0.5315234065055847
Bulletin de la Société de chimie biologique 0.5220468640327454
Methods in Enzymology 0.5183045268058777
New Phytologist 0.516709566116333
Transactions of The Royal Society of South Africa 0.5160568952560425
Biochemical Journal 0.5088347792625427
Journal of Chromatography A 0.5042034983634949
------------------
find neighbors for Nature in 1960's:
2024-11-06 16:51:11 -- start loading word vectors...
2024-11-06 16:51:11 -- word vectors loaded, and its shape is: (3116, 100)
Science 0.6492958068847656
International Geophysics 0.5400177240371704
Naturwissenschaften 0.5398

# Look up periodicals' papers (Paper_detialed_df)

In [17]:
with open('/media/sdb/p2v/pickles/paper_detailed.pkl', 'rb') as file:
    model.paper_detailed_df = pickle.load(file) 

In [19]:
venue_name = 'Database'
start_year = '1990'
end_year = '1999'

print("total number of papers:", len(model.paper_detailed_df[start_year:end_year][(model.paper_detailed_df[start_year:end_year].OriginalVenue==venue_name)]))
model.paper_detailed_df[start_year:end_year][(model.paper_detailed_df[start_year:end_year].
                                              OriginalVenue==venue_name)].sort_values('CitationCount', ascending=False).head(10)

total number of papers: 859


,PaperID,Doi,DocType,PaperTitle,JournalID,ConferenceSeriesID,ReferenceCount,CitationCount,OriginalVenue,VenueID
Year,,,,,,,,,,
1992-01-01,2251843513,0,Journal,an introduction to geographic information systems linking maps to databases,5986019,0,0,98,Database,5986019
1992-01-01,2999636323,10.5555/135779.135781,Journal,an introduction to geographic information systems,5986019,0,0,38,Database,5986019
1999-01-01,2993253906,0,Journal,librarians as knowledge management agents,5986019,0,0,36,Database,5986019
1994-01-01,143070122,0,Journal,adventures with the world wide web creating a hypertext library information system,5986019,0,2,35,Database,5986019
1993-01-01,30281716,0,Journal,the online journal of current clinical trials an innovation in electronic journal publishing,5986019,0,0,25,Database,5986019
1992-01-01,123357955,0,Journal,information manufacturing the road to database quality,5986019,0,0,19,Database,5986019
1992-01-01,2489667996,0,Journal,campaign 92 the campaign library from mead data central,5986019,0,0,19,Database,5986019
1999-01-01,2471229882,0,Journal,knowledge management in the library not,5986019,0,0,16,Database,5986019
1992-01-01,1525851796,0,Journal,an introduction to geographic information systems linking maps to databases and maps for the rest of us affordable and fun,5986019,0,0,16,Database,5986019


In [147]:
VID = 61274580
start_year = '1990'
end_year = '1999'

print("total number of papers:", len(model.paper_detailed_df[start_year:end_year][(model.paper_detailed_df[start_year:end_year].VenueID==VID)]))
model.paper_detailed_df[start_year:end_year][(model.paper_detailed_df[start_year:end_year].
                                              VenueID==VID)].sort_values('CitationCount', ascending=False).head(10)

total number of papers: 1


,PaperID,Doi,DocType,PaperTitle,JournalID,ConferenceSeriesID,ReferenceCount,CitationCount,OriginalVenue,VenueID
Year,,,,,,,,,,
1991-01-01,2498079382,0,Journal,comparative intergenerational stratification research,61274580,0,0,23,Annual Review of Sociology,61274580


In [34]:
focal_paper_list = model.paper_detailed_df[start_year:end_year][model.paper_detailed_df[start_year:end_year].OriginalVenue==venue_name].PaperID.to_list()

# Look up citation trail (random walks)

In [ ]:
focal_walks = []
vid = get_vid('English')
for walk in model.walks:
    if vid in walk:
        focal_walks.append(walk)

In [ ]:
focal_walks

In [ ]:
VID_list = [67764474.0, 67764474.0, 67764474.0, 86431830.0]
[model.MAG_venue_info_df.at[VID,'OriginalVenue'] for VID in VID_list]

# Loop up MAG reference records (ref.pkl)

In [65]:
start_year=1980
end_year=1989

In [31]:
model.load_ref_df(full_load=False, start_year, end_year)

2023-11-08 12:25:48 -- start loading 'ref_df'...
2023-11-08 12:25:49 -- finish.


In [63]:
for paper_ID in focal_paper_list:
    if len(model.target_ref_df[model.target_ref_df.PaperReferenceID == paper_ID]) !=0:
        display(model.target_ref_df[model.target_ref_df.PaperReferenceID == paper_ID])

,PaperReferenceID
PaperID,
1491928743,2315189172


,PaperReferenceID
PaperID,
2781934296,2319931332


,PaperReferenceID
PaperID,
13075566,2328347537
265484110,2328347537
1971602878,2328347537
1981137724,2328347537
1983500773,2328347537
1987161240,2328347537
1993474485,2328347537
2008987134,2328347537
2023797625,2328347537


,PaperReferenceID
PaperID,
2782136301,2313988925


,PaperReferenceID
PaperID,
2781947361,2315566305


,PaperReferenceID
PaperID,
2782425643,2333737838


,PaperReferenceID
PaperID,
2164128878,2014411277


In [85]:
model.paper_detailed_df[model.paper_detailed_df.PaperID==2328347537]

,PaperID,Doi,DocType,PaperTitle,JournalID,ConferenceSeriesID,ReferenceCount,CitationCount,OriginalVenue,VenueID
Year,,,,,,,,,,
1984-01-01,2328347537,10.1093/ENGLISH/33.147.263,Journal,a text book,86431830,0,0,177,English,86431830


In [95]:
model.paper_detailed_df[model.paper_detailed_df.PaperID==1987161240]

,PaperID,Doi,DocType,PaperTitle,JournalID,ConferenceSeriesID,ReferenceCount,CitationCount,OriginalVenue,VenueID
Year,,,,,,,,,,
1984-01-01,1987161240,10.1007/BF02166795,Journal,solvent effects on the37cl n γ 38cl reaction products of some solid chlorobenzene derivatives,63023845,0,6,4,Journal of Radioanalytical and Nuclear Chemistry,63023845
